# Data Cleaning Methodology

## Census API Data Pull

In [4]:
# Import Libraries for API Key
from census import Census
from us import states
import csv
import time
import pdb
import pandas as pd

# Initialize Census object with API key to utilize census python wrapper to pull data
c = Census("a843158c55ae161d6ec8bfe7530da4c8e5acebbc")

The below is an example of a dictionary we will build out that will contain all of the tables
we intend to make request to in the census api. We will leverage the **sub-group** as the key (i.e. B17015_038E) and **description** as the value. Overall, we intend to make request for each of these categories and likely more as the project develops:
- unemployment rate
- Poverty Rate
- Median Household Income rate
- Educational Attainment
- Race/demographics
- Geo-location
- AT HOME FOR THE POPULATION 25 YEARS AND OVER
- Per capita income

In [5]:
# Key is an example of the name that needs to be passed in to get data
tables_dict = {
    'B17015_038E': "POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY SOCIAL SECURITY INCOME BY SUPPLEMENTAL SECURITY INCOME (SSI) AND CASH PUBLIC ASSISTANCE INCOME"
}

The below cell iterates through each key in our `tables_dict`. We will use the key as the name of the csv, referncing the `tables_dict` to remember the data we are using. 

For each key we iterate through, we will be iterating through the years for the ACS5 dataset, which is:

> The 5-year estimates from the ACS are "period" estimates that represent data collected over a period of time.

For each year, we will:
1. Make a get request `c.acs5.state_county` at the state-county level for the specified sub-group (i.e. B17015_038E)
2. We will index into the first object in the response to leverage as out columns
3. Finally, we will iterate through each state-county response and write them to the file

In [7]:
# Years we want to collect. The census api already has this attribute, so we don't errneously
# pull years not available
years = c.acs5.years

for key in tables_dict.keys():
    with open(f'{key}.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile)
        
        # Iterate through years
        for idx, year in enumerate(years):
            
            # Fetch data
            response = c.acs5.state_county(('NAME', key), Census.ALL, Census.ALL, year=year)
            
            # Check if this is the first index so we don't re-write headers each time.
            if idx == 0:
                # Collect columns and write them as the header
                columns = list(response[0].keys()) + ['year']
                spamwriter.writerow(columns)
            
            # Iterate through each nested response and write them to the csv 
            for row in response:
                #pdb.set_trace()
                spamwriter.writerow([row[name] for name in list(response[0].keys())] + [year])
            
            # Delay script every 30 secs so that we don't get flagged for request abuse
            time.sleep(30)

Once all of the data has been collected, we will do some data cleaning. Example of this is shown below, splitting the *state* and *county* from the **NAME** column and placing them into their respective columns.

In addition to the above, we intend to:
- Aggregate *geo-location* and other elements to the tables we create.
- Assess `Null`/`0` values to see if we want to retain them

In [2]:
# Read data in
df = pd.read_csv('B17015_038E.csv')

In [7]:
df.head()

,NAME,B17015_038E,state,county,year
0,"Washington County, Mississippi",497.0,28,151,2018
1,"Perry County, Mississippi",124.0,28,111,2018
2,"Choctaw County, Mississippi",46.0,28,19,2018
3,"Itawamba County, Mississippi",212.0,28,57,2018
4,"Carroll County, Mississippi",41.0,28,15,2018


In [12]:
# Split and pass into columns
df['county'] = df['NAME'].str.split(',').str[0]
df['state'] = df['NAME'].str.split(',').str[1]
df.drop('NAME', inplace=True, axis=1)

In [20]:
df.head()

,B17015_038E,state,county,year
0,497.0,Mississippi,Washington County,2018
1,124.0,Mississippi,Perry County,2018
2,46.0,Mississippi,Choctaw County,2018
3,212.0,Mississippi,Itawamba County,2018
4,41.0,Mississippi,Carroll County,2018


In [19]:
# Identify how many 0 values exist in df
df[df['B17015_038E'] == 0].shape

(235, 4)